In [ ]:
!pip install simpy
!pip install xlrd
!pip install openpyxl


You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 96 kB 6.4 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 243 kB 24.5 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import simpy
import itertools
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
termin_info = pd.read_excel("Data4SVONEW.xlsx", sheet_name=0)
termin_info.columns = termin_info.columns.str.replace(' ', '')
ctt_inf = pd.read_excel("Data4SVONEW.xlsx", sheet_name=1)
termin_info

,AirportID,TerminalID,StructureID,ProcessingTime,QueryCapacity,Capacity,TimeToRestart,Unnamed:7
0,SVO,B,BHS,240.000000,40.000000,200.000000,10.000000,NaN
1,SVO,B,INBP,120.000000,40.000000,200.000000,10.000000,NaN
2,SVO,B,OUTBP,120.000000,40.000000,200.000000,10.000000,NaN
3,SVO,B,TBS,120.000000,40.000000,1000.000000,10.000000,NaN
4,SVO,B,LTTBS,240.000000,40.000000,10000.000000,10.000000,NaN
5,SVO,C,BHS,240.000000,40.000000,200.000000,10.000000,NaN
6,SVO,C,INBP,120.000000,40.000000,200.000000,10.000000,NaN
7,SVO,C,OUTBP,120.000000,40.000000,200.000000,10.000000,NaN
8,SVO,C,TBS,120.000000,40.000000,1000.000000,10.000000,NaN
9,SVO,C,LTTBS,240.000000,40.000000,10000.000000,10.000000,NaN


In [ ]:
ctt_inf

,AirportID,TerminalDepID,TerminalArrID,ProcessingTime,QueryCapacity,Capacity,TimeToRestart
0,SVO,D,E,120,20,100,10
1,SVO,E,D,120,20,100,10
2,SVO,E,B,360,10,100,10
3,SVO,B,E,120,20,100,10
4,SVO,F,E,120,20,100,10
5,SVO,E,F,120,20,100,10
6,SVO,C,B,120,20,100,10
7,SVO,B,C,120,20,100,10


In [ ]:
class my_class(object):
    def __init__(self, sid, p_t, q_cap, capacity, ResT):
        self.p_t, self.ResT, self.capacity, self.sid, self.q_cap = p_t, ResT, capacity, sid, q_cap

class TermL(object):
    def __init__(self, terminal_id, BHS, INBP, OUTBP, TBS, LTTBS):
        self.BHS, self.terminal_id, self.INBP, self.TBS, self.LTTBS, self.OUTBP = BHS, terminal_id, INBP, TBS, LTTBS, OUTBP

def get_structures(terminal_id, df): 
    df = df[df['TerminalID']==terminal_id]
    terms = []
    for sid in ['BHS', 'INBP', 'OUTBP',
                         'TBS', 'LTTBS']:
        valS = df.loc[df['StructureID']==sid, ['ProcessingTime',
                                                           'QueryCapacity',
                                                           'Capacity']].values[0]
        terms.append(my_class(sid, valS[0], valS[1], valS[2], 5))
    return terms

class CTT(object):
    def __init__(self, dep, arr, p_t, q_cap, capacity, ResT):
        self.q_cap, self.p_t, self.ResT, self.capacity, self.dep, self.arr = q_cap, p_t, ResT, capacity, dep, arr

def get_terminals(df):
    term_all = []
    for terminal_id in df['TerminalID'].unique(): 
        terminal_ = TermL(terminal_id, *get_structures(terminal_id, df))
        term_all.append(terminal_)
    return term_all

term_all = get_terminals(termin_info)

In [ ]:
paisterm = [] 
for p1 in ['B', 'C', 'D', 'E']:
     for p2 in  ['B', 'C', 'D', 'E'] :
         print("'",p1+p2, "'", sep="")


'BB'
'BC'
'BD'
'BE'
'CB'
'CC'
'CD'
'CE'
'DB'
'DC'
'DD'
'DE'
'EB'
'EC'
'ED'
'EE'


In [ ]:
CTT_NS = CTT("N", "S", 360, 20, 100, 10)
CTT_SN = CTT("S", "N", 360, 20, 100, 10)

CTT_BN = CTT("B", "N", 0, 100000000, 100000000, 0)
CTT_NB = CTT("N", "B", 0, 100000000, 100000000, 0)
CTT_CN = CTT("C", "N", 0, 100000000, 100000000, 0)
CTT_NC = CTT("N", "C", 0, 100000000, 100000000, 0)

CTT_DS = CTT("D", "S", 0, 100000000, 100000000, 0)
CTT_SD = CTT("S", "D", 0, 100000000, 100000000, 0)
CTT_ES = CTT("E", "S", 0, 100000000, 100000000, 0)
CTT_SE = CTT("S", "E", 0, 100000000, 100000000, 0)



CTT_dict = {"NS": CTT_NS, "SN": CTT_SN,
             'BN': CTT_BN, 'NB': CTT_NB, 'CN': CTT_CN, 'NC': CTT_NC,
             'DS': CTT_DS, 'SD': CTT_SD, 'ES': CTT_ES, 'SE': CTT_SE}

CTT_convert = {
    'BC': 'BN', 'NC': 'NC', 
    'CB': 'CN', 'NB': 'NB',
    'DE': 'DS', 'SE': 'SE',
    'ED': 'ES', 'SD': 'SD',
    'BD': 'BN', 'ND': 'NS', 'SD': 'SD',
    'BE': 'BN', 'NE': 'NS', 'SE': 'SE',
    'CD': 'CN', 'ND': 'NS', 'SD': 'SD',
    'CE': 'CN', 'NE': 'NS', 'SE': 'SE',
    'DB': 'DS', 'SB': 'SN', 'NB': 'NB',
    'EB': 'ES', 'SB': 'SN', 'NB': 'NB',
    'DC': 'DS', 'SC': 'SN', 'NC': 'NC',
    'EC': 'ES', 'SC': 'SN', 'NC': 'NC'}

In [ ]:
#lug_d = pd.read_csv("result2019.csv") # 69420
lug_d = pd.read_csv("debug_set.csv") #delete

lug_d.drop('Unnamed: 0', axis=1, inplace=True)
lug_d = lug_d[(lug_d.isTransit == 1) & (lug_d.DepartureTerminal != 'F') & (lug_d.ArrTerminal != 'F')].drop_duplicates(subset='TagID', keep="last")
lug_d

,TagID,AirportID,isTransit,ArrFlNum,DepFlNum,ArrDateTime,DepDateTime,ArrTerminal,DepartureTerminal,Status
0,1,SVO,1,SU1625,SU2060,22.12.19 20:40:00,23.12.19 09:55:00,B,C,0
1,2,SVO,1,LO677,SU1408,22.12.19 20:40:00,23.12.19 00:30:00,E,B,0
2,3,SVO,1,SU1359,SU1166,22.12.19 20:40:00,23.12.19 02:30:00,B,B,0
3,4,SVO,1,SU1359,SU1166,22.12.19 20:40:00,23.12.19 02:30:00,B,D,0
4,5,SVO,1,SU2357,SU1306,22.12.19 20:45:00,23.12.19 11:05:00,E,B,0
5,6,SVO,1,SU1331,SU1106,30.12.19 21:15:00,30.12.19 23:15:00,B,B,0
6,7,SVO,1,SU2357,SU1852,30.12.19 21:15:00,30.12.19 23:15:00,E,B,0
7,8,SVO,1,SU1375,SU1522,30.12.19 21:15:00,30.12.19 23:15:00,B,B,0
8,9,SVO,1,SU1375,SU1382,30.12.19 21:15:00,30.12.19 23:15:00,B,B,0
9,10,SVO,1,SU1375,SU100,30.12.19 21:15:00,30.12.19 23:15:00,B,B,0


In [ ]:
l_in_d = lug_d.copy()
l_in_d[["ArrDateTime", "DepDateTime"]] = l_in_d[["ArrDateTime", "DepDateTime"]].apply(pd.to_datetime, format='%d.%m.%y %H:%M:%S')
l_in_d["ArrDateTime"], l_in_d["DepDateTime"] = np.where(l_in_d["ArrDateTime"] > l_in_d["DepDateTime"],
 [l_in_d["DepDateTime"], l_in_d["ArrDateTime"]], [l_in_d["ArrDateTime"], l_in_d["DepDateTime"]])
l_in_d = l_in_d.sort_values(by=['ArrDateTime'])
l_in_d

,TagID,AirportID,isTransit,ArrFlNum,DepFlNum,ArrDateTime,DepDateTime,ArrTerminal,DepartureTerminal,Status
0,1,SVO,1,SU1625,SU2060,2019-12-22 20:40:00,2019-12-23 09:55:00,B,C,0
1,2,SVO,1,LO677,SU1408,2019-12-22 20:40:00,2019-12-23 00:30:00,E,B,0
2,3,SVO,1,SU1359,SU1166,2019-12-22 20:40:00,2019-12-23 02:30:00,B,B,0
3,4,SVO,1,SU1359,SU1166,2019-12-22 20:40:00,2019-12-23 02:30:00,B,D,0
4,5,SVO,1,SU2357,SU1306,2019-12-22 20:45:00,2019-12-23 11:05:00,E,B,0
43,44,SVO,1,SU1249,SU1108,2019-12-30 21:15:00,2019-12-30 23:15:00,B,B,0
42,43,SVO,1,SU1249,SU100,2019-12-30 21:15:00,2019-12-30 23:15:00,B,B,0
41,42,SVO,1,SU1249,SU100,2019-12-30 21:15:00,2019-12-30 23:15:00,B,B,0
40,41,SVO,1,SU1249,SU100,2019-12-30 21:15:00,2019-12-30 23:15:00,B,B,0
39,40,SVO,1,SU1249,SU2402,2019-12-30 21:15:00,2019-12-30 23:15:00,B,B,0


In [ ]:
if False: 
    l_in_d = l_in_d[l_in_d.ArrTerminal != l_in_d.DepartureTerminal]

In [ ]:
class Sim_class(object):
    def __init__(self, name, main_resource, buffer_resource, q_r, p_t, ResT):
        self.q_r, self.name, self.ResT, self.full_cap, self.full_q, self.buffer_resource, self.p_t, self.main_resource = q_r, name, ResT, False, False, buffer_resource, p_t, main_resource

def cont_iter(on_term, name, main_info):
    if name == 'CTT': return str_dic[(on_term, 'CTT_'+CTT_convert[on_term + main_info['DepartureTerminal']])]
    else: return str_dic[(on_term, name)]

def ins_entry(event_type, terminal, system, tag_id=0, FlNum="NA", time=None):
    if time != None: datetime = time
    else: datetime = start_time + timedelta(seconds=float(env.now))
    row = { 'eventTypeID': event_type, 'TagID': tag_id, 'TerminalID': terminal, 'SystemID': system, 'FlNum': FlNum, 'DateTime': datetime}
    logs_to_save.append(row)

In [ ]:
def next_pos(env, tag_id, on_term, name):
    main_info = all_bag_dic[tag_id]
    New_it = cont_iter(on_term, name, main_info)
    struct_info = len(New_it.q_r.queue) > 0 and not New_it.full_q
    if struct_info or struct_info == None:
        New_it.full_q = True 
        ins_entry('M001', on_term, New_it.name, tag_id=tag_id)
    queue_req = New_it.q_r.request()
    ins_entry('B002', on_term, New_it.name, tag_id=tag_id)
    yield queue_req
    if len(New_it.q_r.queue) == 0 and New_it.full_q: 
        New_it.full_q = False
        ins_entry('M002', on_term, New_it.name, tag_id=tag_id)
    next_req = New_it.main_resource.request()
    yield next_req
    buffer_req = New_it.buffer_resource.request()
    yield buffer_req
    New_it.q_r.release(queue_req)
    yield env.timeout(New_it.ResT)
    New_it.buffer_resource.release(buffer_req)
    ins_entry('B001', on_term, New_it.name, tag_id=tag_id)
    yield env.timeout(New_it.p_t)
    if name in ['LTTBS', 
                'TBS']:
        if New_it.main_resource.count == New_it.main_resource.capacity and not New_it.full_cap:
            New_it.full_cap = True 
            ins_entry('M003', on_term,
                     New_it.name,
                     tag_id=tag_id)
        elif New_it.main_resource.count < New_it.main_resource.capacity and New_it.full_cap:
            New_it.full_cap = False
            ins_entry('M004', 
                     on_term, 
                     New_it.name,
                     tag_id=tag_id)
        await_time = main_info['DepDateTime'] - start_time - timedelta(seconds=float(env.now)) - timedelta(hours=2)
        if await_time.total_seconds() > 0: yield env.timeout(await_time.seconds)
    New_it.main_resource.release(next_req)

In [ ]:
def do_bagging(env, tag_id):
    now_on = 'aircraft'
    main_info = all_bag_dic[tag_id]
    on_term = main_info['on_term']
    while 1:
        if now_on == 'aircraft': now_on = 'INBP'
        elif now_on == 'INBP': now_on = 'BHS'
        elif now_on == 'BHS':
            if on_term == main_info['DepartureTerminal']:
                if main_info['DepDateTime'] <= (start_time + timedelta(seconds=float(env.now)) + timedelta(hours=2) ): now_on = 'OUTBP'
                elif main_info['DepDateTime'] <= (start_time + timedelta(seconds=float(env.now)) + timedelta(hours=4) ): now_on = 'TBS'
                else: now_on = 'LTTBS'
            else: now_on = 'CTT'
        elif now_on in ['CTT', 'TBS', 'LTTBS']: now_on = 'BHS'
        elif now_on == 'OUTBP': break
        yield env.process(next_pos(env, tag_id, on_term, name=now_on))
        if now_on == 'CTT': on_term = CTT_convert[on_term + main_info['DepartureTerminal']][-1]

    if main_info['DepDateTime'] < start_time + timedelta(seconds=float(env.now)):
        ins_entry('M005', on_term, 'NA', tag_id=tag_id, FlNum=main_info['DepFlNum'])
        diff = start_time + timedelta(seconds=float(env.now)) - main_info['DepDateTime']
        laters.append(diff)
        case_late(diff, main_info)
    else:
        ins_entry('B003', on_term, 'NA', tag_id=tag_id, FlNum=main_info['DepFlNum'])
        ins_entry('B004', on_term, 'NA', tag_id=tag_id, FlNum=main_info['DepFlNum'])

In [ ]:
late_enties = [] 
def case_late(diff, main_info):
    global late_enties
    curr_loss = {   "DepDateTime":  main_info['DepDateTime'],
                    "ArrDateTime": main_info['ArrDateTime'],
                    "ArrTerminal": main_info["ArrTerminal"],
                    "DepartureTerminal": main_info["DepartureTerminal"],
                    "TimeDifference": diff,
                    "TagID": main_info['TagID'],
                     "ArrFlNum": main_info['ArrFlNum'],
                    "DepFlNum": main_info['DepFlNum']}

    late_enties.append(curr_loss)

In [ ]:
def pack_bag(df):
    for index, row in df.iterrows():
        if row['ArrDateTime'] > start_time + timedelta(seconds=float(env.now)): yield env.timeout((row['ArrDateTime'] - start_time - timedelta(seconds=float(env.now))).seconds)   
        if (row['ArrFlNum'], row['ArrDateTime']) not in procc_arr:
            procc_arr.add((row['ArrFlNum'], row['ArrDateTime']))
            ins_entry('A001', row['ArrTerminal'], 'NA', FlNum=row['ArrFlNum'])
        if (row['DepFlNum'], row['DepDateTime']) not in procc_dep:
            procc_dep.add((row['DepFlNum'], row['DepDateTime']))
            ins_entry('A002', row['DepartureTerminal'], 'NA', FlNum=row['DepFlNum'], time=row['DepDateTime'])
        row['on_term'] = row['ArrTerminal']
        all_bag_dic[row['TagID']] = row
        env.process(do_bagging(env, row['TagID']))

In [ ]:
env = simpy.Environment()
str_dic = {}
logs_to_save = []
all_bag_dic = {}
start_time = l_in_d['ArrDateTime'].min()
procc_arr = set()
procc_dep = set()
laters = []


for t in term_all:
    for s in [t.INBP, t.BHS,
                   t.TBS, t.LTTBS, t.OUTBP]:
        str_dic[(t.terminal_id,
                        s.sid)] = Sim_class(
            name=s.sid, 
            main_resource=simpy.Resource(env, 
                                         capacity=s.capacity), 
            buffer_resource=simpy.Resource(env, capacity=1), 
            q_r=simpy.Resource(env, 
                                          capacity=s.q_cap),
            p_t=s.p_t,
            ResT=s.ResT,
        )
for key, val in CTT_dict.items():
    str_dic[(key[0], f'CTT_{key}')] = Sim_class(
            name=f'CTT_{key}',
            main_resource=simpy.Resource(env, 
                                         capacity=val.capacity), 
            buffer_resource=simpy.Resource(env, capacity=1), 
            q_r=simpy.Resource(env,
                                          capacity=val.q_cap),
            p_t=val.p_t,
            ResT=val.ResT,
    )
env.process(pack_bag(l_in_d))
env.run()

logs_df = pd.DataFrame(logs_to_save)
logs_df.sort_values(['DateTime', 'TagID'], inplace=True, ignore_index=True)
logs_df['eventID'] = logs_df.index + 1
cols = list(logs_df.columns)
cols = [cols[-1]] + cols[:-1]
logs_df = logs_df[cols]

In [ ]:
def inst(x):
    i = 10
    return x[:i] +"T"+x[i+1:]

In [ ]:

logs_df['DateTime'] = logs_df['DateTime'].apply(str)
logs_df['DateTime'] = logs_df['DateTime'].apply(inst)

In [ ]:
logs_df = logs_df[['eventID', 'TagID',
 'DateTime',
 'SystemID',
 'eventTypeID',
 'TerminalID',
 'FlNum']]

In [ ]:
#logs_df = logs_df.rename({'TerminalID':'Terminal'})


In [ ]:
oldn = ['BHS', 'INBP', 'OUTBP', 'TBS', 'LTTBS', 'CTT']
newn = ['SORTER', "TRIN", "CAR", "EBS", "LBS", "STATION"]
logs_df = logs_df.replace(oldn, newn)

In [ ]:
logs_df.TagID = logs_df.TagID.replace(0, ' ')

In [ ]:
mask = ( (logs_df.TerminalID == 'S') | (logs_df.TerminalID == 'N'))  & (logs_df.SystemID == "SORTER")
logs_df.loc[mask, 'SystemID'] = 'STATION'

In [ ]:
to_del = ['CTT_BN', 'CTT_NB',  'CTT_CN', 'CTT_NC', 'CTT_DS', 'CTT_SD', 'CTT_ES',  'CTT_SE']
maskdrop1 = logs_df.SystemID.isin(to_del)
logs_df = logs_df.drop(logs_df.loc[maskdrop1].index)

In [ ]:
mask1 = (logs_df.SystemID.isin(['CTT_NS', 'CTT_SN']) )
logs_df.loc[mask1, 'TerminalID'] = 'NA'

In [ ]:
logs_df.loc[logs_df.shape[0]:logs_df.shape[0]]

,eventID,TagID,DateTime,SystemID,eventTypeID,TerminalID,FlNum
2517,2518,21,2019-12-30T21:23:10,NA,B003,B,SU2658


In [ ]:
arr_dep_diff = procc_arr - procc_dep
arr_dep_diff = list(arr_dep_diff)
arr_dep_diff = sorted(arr_dep_diff, key= lambda x: x[1])
#arr_dep_diff

In [ ]:
# l_in_d


In [ ]:
maskNS = logs_df.SystemID.isin(['CTT_NS'])
maskSN = logs_df.SystemID.isin(['CTT_SN']) 
logs_df.loc[maskNS, 'TerminalID'] = 'N-S'
logs_df.loc[maskSN, 'TerminalID'] = 'S-N'
logs_df.loc[maskNS, 'SystemID'] = 'CTT'
logs_df.loc[maskSN, 'SystemID'] = 'CTT'

In [ ]:
logs_df.TagID = logs_df.TagID.replace(' ', "NA")

In [ ]:
# logs_df[logs_df.eventTypeID != 'B002']

In [ ]:
logs_df

,eventID,TagID,DateTime,SystemID,eventTypeID,TerminalID,FlNum
0,1,NA,2019-12-22T20:40:00,NA,A001,B,SU1625
1,2,NA,2019-12-22T20:40:00,NA,A001,E,LO677
2,3,NA,2019-12-22T20:40:00,NA,A001,B,SU1359
3,4,1,2019-12-22T20:40:00,TRIN,B002,B,NA
4,5,2,2019-12-22T20:40:00,TRIN,B002,E,NA
...,...,...,...,...,...,...,...
2612,2613,NA,2019-12-30T23:15:00,NA,A002,B,SU2454
2613,2614,NA,2019-12-30T23:15:00,NA,A002,B,SU2658
2614,2615,NA,2019-12-30T23:15:00,NA,A002,B,SU1234
2615,2616,NA,2019-12-30T23:15:00,NA,A002,B,SU1052


In [ ]:
logs_df.to_csv("goodlogs.csv", sep=';',  index=False)

In [ ]:
logs_df.to_csv("logsdemo.csv", sep=';',  index=False)

In [ ]:
logs_df

,eventID,TagID,DateTime,SystemID,eventTypeID,TerminalID,FlNum
0,1,NA,2019-12-22T20:40:00,NA,A001,B,SU1625
1,2,NA,2019-12-22T20:40:00,NA,A001,E,LO677
2,3,NA,2019-12-22T20:40:00,NA,A001,B,SU1359
3,4,1,2019-12-22T20:40:00,TRIN,B002,B,NA
4,5,2,2019-12-22T20:40:00,TRIN,B002,E,NA
...,...,...,...,...,...,...,...
2612,2613,NA,2019-12-30T23:15:00,NA,A002,B,SU2454
2613,2614,NA,2019-12-30T23:15:00,NA,A002,B,SU2658
2614,2615,NA,2019-12-30T23:15:00,NA,A002,B,SU1234
2615,2616,NA,2019-12-30T23:15:00,NA,A002,B,SU1052


In [ ]:
# pd.read_csv("bench.csv", sep=';').fillna('NA')


,eventID,TagID,DateTime,SystemID,eventTypeID,TerminalID,FlNum
0,1,NA,2019-12-22T20:40:00,NA,A001,B,SU1625
1,2,NA,2019-12-22T20:40:00,NA,A001,E,LO677
2,3,NA,2019-12-22T20:40:00,NA,A001,B,SU1359
3,4,1.0,2019-12-22T20:40:00,TRIN,B002,B,NA
4,5,2.0,2019-12-22T20:40:00,TRIN,B002,E,NA
...,...,...,...,...,...,...,...
2512,2613,NA,2019-12-30T23:15:00,NA,A002,B,SU2454
2513,2614,NA,2019-12-30T23:15:00,NA,A002,B,SU2658
2514,2615,NA,2019-12-30T23:15:00,NA,A002,B,SU1234
2515,2616,NA,2019-12-30T23:15:00,NA,A002,B,SU1052


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=31539d55-3766-40bf-bd6a-cd67a311cbbb' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>